# Aula 9 - Parte 1 - Processamento de Linguagem Natural

Esse notebook descreve o passo a passo a ser aplicado para o Processamento de Linguagem Natural para criar uma análise de sentimento em comentários sobre o Youtube.

In [ ]:
from time import time

In [ ]:
ti = time()

In [ ]:
# Essa biblioteca realiza o tratamento da linguagem natural 
# com diversas ferramentas disponíveis (NLTK - Natural Language Tookit)
import nltk

In [ ]:
# Necessário realizar o download das palavras StopWords
nltk.download('stopwords')

In [ ]:
# Lematizador para palavras em Portugues
nltk.download('rslp')

## 1. Carregar bases de dados

A base de dados está salvo em formato de CSV. Onde o primeiro elemento é o texto e o segundo elemento é a classe.

 | texto | classe |
 | --- | --- |
 |0	|@pandlrcom Quem é que liga pra copa gente? Pelo o amor de Deus	|negativo
|1	|Faz a seleção aí do teu time — eu e a carol no ataque, Thaynara e veve na zaga, Luise no gol!!! É esse? kkk	|neutro
|2|	Cristiano Ronaldo com grife, 78 milhões de euros 😎		|positivo


Iremos estruturar os dados para facilitar todo o processo de criação do algoritmo de análise de sentimento.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
def carregar_dados(arquivo):
    dados = pd.read_csv(arquivo, sep=';', encoding='utf-8')
    
    base = []
    for i in range(len(dados)):
        base.append((dados.texto.loc[i], dados.classe.loc[i]))
    return base

In [ ]:
base_treino = carregar_dados('dados_treino.csv')

In [ ]:
base_treino[0]

In [ ]:
base_teste = carregar_dados('dados_teste.csv')

In [ ]:
base_teste[0]

In [ ]:
print("Tamanho base de treino: ", len(base_treino))
print("Tamanho base de teste: ", len(base_teste))

In [ ]:
print(base_treino[0])

## 2. Pré Processamento

Esta primeira etapa tem o propósito de realizar o pré-processamento dos dados, necessário para a preparação da base de dados para algoritmos de aprendizagem de máquina.

Essa etapa envolve os seguintes passos:

    2.1 - Remoção de pontuação, deixar tudo minúsculo e remoção de URLs, RT e @user
    2.2 - Remoção de Stopwords
    2.3 - Remoção do radical das palavras (Stemming)
    2.4 - Listagem de todas as palavras da base
    2.5 - Extração de palavras únicas
    2.6 - Junção de palavras únicas
    2.7 - Extração de palavras de cada frase


### 2.1 - Remoção de pontuação, deixar tudo minúsculo e remoção de URLs, RT e @user

O objetivo dessa etapa é realizar um tratamento no texto removendo as pontuações, urls, RTs, menção a usuários e também padronizar toda a frase e minúscula.

In [ ]:
import re

In [ ]:
def tratar_texto(texto):
    string_sem_url = re.sub(r"http\S+", "", str(texto))
    string_sem_user = re.sub(r"@\S+", "", str(string_sem_url))
    string_sem_rt = re.sub(r"RT+", "", str(string_sem_user))
    return str(string_sem_rt).strip()

In [ ]:
tratar_texto('Esse link é ótimo http://ashdfasdfa.com')

In [ ]:
tratar_texto('RT @user a copa está demais')

In [ ]:
tratar_texto('@user a copa está demais')

In [ ]:
tratar_texto('RT a copa está demais')

In [ ]:
import string


In [ ]:
def remover_pontuacao(base):
    """Essa função remove as pontuações da base.
    Args:
        base: contém todos tweets no formato (texto,classe).
    Returns:
        base_dados: É uma lista de tuplas.
    """
    frases_final = []
    for (frase, classe) in base:
        sem_pontuacao = []
        # Para cada palavra na frase
        frase = tratar_texto(frase)
        for p in frase:
            # Verifica se não é uma pontuação
            if p not in string.punctuation:
                sem_pontuacao.append(p)
        # Refaz a frase
        aux = ''.join(sem_pontuacao)
        # Salva na lista final no formato (texto,classe)
        frases_final.append((aux.lower(), classe))
    # Retorna todo o conjunto sem as pontuações
    return (frases_final)


In [ ]:
print(base_treino[0])

In [ ]:
print(remover_pontuacao([base_treino[0]]))

In [ ]:
print(string.punctuation)

In [ ]:
frases_sem_pontuacao = remover_pontuacao(base_treino)

In [ ]:
print(frases_sem_pontuacao[0])

### 2.2 - Remoção de Stopwords

As stopwords são palavras que não possuem significado para o desempenho dos algoritmos de classificação de texto. Por exemplo: 'é', 'muito', 'o', 'por', entre outros.

A permanência delas na base de dados, pode provocar maior lentidão no processamento dos dados, sem utilidade para o contexto em que estamos trabalhando.

A função ```remover_stopwords``` funciona da seguinte forma:

- O parâmetro "frases_sem_pontuacao" representa toda a base de dados já tratada.
- Essa base de dados contém um par de elementos sendo o primeiro a frase e o segundo a classe (positivo, negativo, neutro).
- A função percorre toda a base de dados, linha a linha, verificando em quais frases possuem as stopwords definidas na biblioteca NLTK. Uma vez identificadas, elas são removidas da frase. 
- O conjunto final é uma estrutura contendo a frase (sem as stopwords) e a classe (positivo, negativo, neutro).

In [ ]:
# Carrega as stopwords definidas na biblioteca para o idioma Português
stopwordsnltk = nltk.corpus.stopwords.words('portuguese')

# Adiciona novas stopwords
stopwordsnltk.append('vou')
stopwordsnltk.append('tão')

# Visualiza algumas
print(stopwordsnltk[:10])

In [ ]:
def remover_stopwords(frases_sem_pontuacao):
    """Essa função remove as stopwords da base.
    Args:
        frases_sem_pontuacao: É uma lista de tuplas.
    Returns:
        frases_final: É uma lista de tuplas (texto, classe).
    """
    stopwordsnltk = nltk.corpus.stopwords.words('portuguese')
    frases_final = []
    for (frase, classe) in frases_sem_pontuacao:
        sem_stop = []
        for palavra in frase.split():
            if palavra not in stopwordsnltk:
                sem_stop.append(palavra)  
        frases_final.append((sem_stop, classe))
    return frases_final




In [ ]:
# Variável que armazena o resultado da função remover_stopwords
frases_sem_stopwords = remover_stopwords(frases_sem_pontuacao)



In [ ]:
print(frases_sem_stopwords[0])

### 2.3 - Remoção do radical das palavras (Stemming)

Stemming é uma técnica utilizada para reduzir a dimensionalidade dos dados na etapa de pré-processamento. É baseada na redução de palavras em seu morfema, de acordo com as regras do idioma que o algoritmo será executado. 

Por exemplo, em português a palavra “casa” possui o morfema “cas” e suas variações: casinhas, casebre, casona.

A função ```aplicar_stemmer``` funciona da seguinte forma:

- Utiliza-se uma ferramenta da biblioteca NLTK para realizar essa técnica. Para isso, acessamos o pacote ```stem``` para realizar essa tarefa.
- ```nltk.stem.RSLPStemmer()``` indica que será utilizado a lingua portguesa.
- O parâmetro ```frases_sem_stopwords``` da função ```aplicar_stemmer``` representa a base de dados sem as stopwords que foram removidas anteriormente.

- A função percorre toda a base de dados aplicando o método ```stemmer.stem(palavra)``` em cada palavra de cada frase, cuja finalidade é deixar apenas o radical de cada palavra.

- Exemplo: A frase ```('eu sou admirada por muitos','positivo')```, após a função ```aplicar_stemmer``` ficará ```(['admir', 'muit'], 'positivo')```

Uma desvantagem da aplicação do algoritmo stemmer é quando duas palavras com sentidos diferentes possuem o mesmo radical, como por exemplo as palavras ```novamente``` e ```novo``` que possuem o radical ```nov``` dessa forma, na etapa de aprendizado de máquina o algoritmo pode perder algumas informações.

In [ ]:
def aplicar_stemmer(frases_sem_stopwords):
    """Função que reduz a palavra ao seu radical
    Args:
        frases_sem_stopwords: lista de tuplas.
    Returns:
        frases_stemming: lista de tuplas.
    """
    stemmer = nltk.stem.RSLPStemmer()
    frases_stemming =[]
    for (frase, classe) in frases_sem_stopwords:
        com_stemming = []
        # Para cada palavra na frase, aplicar o stemmer e salvar
        for palavra in frase:
            com_stemming.append(str(stemmer.stem(palavra)))
        frases_stemming.append((com_stemming, classe))    
    # Retornar todo o conjunto com o stemming aplicado
    return frases_stemming


In [ ]:
frases_com_stemmer = aplicar_stemmer(frases_sem_stopwords)

In [ ]:
print(frases_com_stemmer[0])

### 2.4 - Listagem de todas as palavras da base

A função ```extrair_palavras``` irá gerar uma nova lista com todas as palavras que já foram pré-processadas anteriormente porém sem a sua classificação (positivo, negativo e neutro) associada.

Funciona da seguinte forma:
- O parâmetro da função representa a lista gerada pela função aplicar_stemmer, que é a ```frases_com_stemmer```.
- Ela percorre toda a base e insere em uma lista com todas as palavras da base de dados, mas sem sua classificação associada.

In [ ]:
def extrair_palavras(frases_com_stemmer):
    """Função que unifica todas as palavras do conjunto de dados em uma única lista.
    Args:
        frases_com_stemmer: Frases com o Stemmer já aplicados.
    Returns:
        todas_palavras: lista com todas as palavras.
    """
    todas_palavras = []
    for (palavras, classe) in frases_com_stemmer:
        todas_palavras.extend(palavras)
    return todas_palavras



In [ ]:
palavras_sem_classe = extrair_palavras(frases_com_stemmer)

In [ ]:
palavras_sem_classe[:10]

### 2.5 - Extração de palavras únicas

Na função ```aplicar_frequencia``` iremos remover os radicais repitidos da base para otimizar o processamento dos dados utilizando o recurso do nltk ```FreqDist```.

A classe ```FreqDist``` unifica todas as palavras repetidas gerando um dicionário do tipo ```chave,valor``` dentro de uma lista, sendo a chave o radical e o valor a frequencia com que ele se repete. Ex: ```('am', 4)```. Nesse exemplo o radical ```am``` apareceu na base de dados 4 vezes.

In [ ]:
def aplicar_frequencia(palavras_sem_classe):
    """Função que aplica a frequencia das palavras
    Args:
        palavras_sem_classe: palvras sem a classificação.
    Returns:
        palavras: FreqDist
    """
    palavras = nltk.FreqDist(palavras_sem_classe)
    return palavras


In [ ]:
frequencia_palavras = aplicar_frequencia(palavras_sem_classe)

In [ ]:
frequencia_palavras

In [ ]:
# Visualizar as 50 frases mais completas
print(frequencia_palavras.most_common(50))



### 2.6 - Junção de palavras únicas

Além disso, temos que criar uma estrutura apenas com as palavras únicas da frequência gerada anteriormente.

In [ ]:
def extrair_palavras_unicas(frequencia_palavras):
    """Função que retorna as palavras únicas
    Args:
        frequencia_palavras: dicionário com a frequencia das palavras.
    Returns:
        freq: palavras unicas.
    """
    freq = frequencia_palavras.keys()
    return freq


In [ ]:
palavras_sem_repeticao = extrair_palavras_unicas(frequencia_palavras)

In [ ]:
# Para visualizar as 5 primeiras palavras é necessário realizar uma conversão direta para o tipo lista
print(list(palavras_sem_repeticao)[:5])


In [ ]:
# A estrutura criada é do tipo dict_keys, que representam as chaves de um dicionário
print(type(palavras_sem_repeticao))


### 2.7 - Extração de palavras de cada frase

O objetivo da função ```criar_caracteristicas``` é auxiliar a caracterização das frases a serem utilizadas no algoritmo Naive Bayes.

O método ```nltk.classify.apply_features``` realiza essa caracterização através do mapeamento de cada frase na função ```criar_caracteristicas```. O resultado desse mapeamento é um dicionário para cada frase onde as palavras que pertencem a respectiva frase sejam ```True```. Todas as outras palavras da base que não pertencem a frase serão definidas como ```False```.

O método ```nltk.classify.apply_features``` exige dois parâmetros, sendo o primeiro uma função que irá extrair as caracteristicas e o segundo é o conjunto de dados onde será aplicado essa caracterização.

Essa etapa é necessária para a preparação da base de dados para o algoritmo de aprendizagem de máquina Naive Bayes. É o resultado dessa função que irá ser passada como parâmetro para criar o classificador.

A função ```criar_caracteristicas``` recebe a lista com as palavras com o stemming e cria uma estrutura onde apenas as palavras que estão nessa lista será marcada como ```True```, todas as outras serão marcadas como ```False```.

In [ ]:
def criar_caracteristicas(documento):
    """Função que cria as características do documento, verificando se a palavra existe ou não no documento.
    Args:
        documento: lista com todas as palavras
    Returns:
        caracteristicas: dicionário com as características.
    """
    global palavras_sem_repeticao
    doc = set(documento)
    caracteristicas = {}
    # Para cada palavra
    for palavra in palavras_sem_repeticao:
        # Se a palavra existir no documento é atribuido True, caso contrário False.
        caracteristicas[palavra] = (palavra in doc)    
    # Listar com as caracteristicas da palavra
    return caracteristicas



O código abaixo é apenas para testar e validar a função ```criar_caracteristicas```.

In [ ]:
# Para testar a função acima, será utilizado duas frases que já foi aplicado o stemmer.
teste_caracteristica = frases_com_stemmer[0:2]


In [ ]:
print(teste_caracteristica)

In [ ]:
print(extrair_palavras(teste_caracteristica))

A execução ```nltk.classify.apply_features``` irá gerar uma lista, onde cada elemento dessa lista é uma tupla com dois elementos, sendo o primeiro o dicionário gerado pela função ```criar_caracteristicas``` e o segundo elemento com a classificação (positivo, negativo, neutro).

In [ ]:
frases_teste_final = nltk.classify.apply_features(criar_caracteristicas, teste_caracteristica)

In [ ]:
# Visualizar 1 elemento classificado
print(frases_teste_final[0])

Para facilitar a reutilização de todo o código criado anteriormente, iremos criar uma função para estruturar qualquer texto que desejamos classificar.

In [ ]:
def estruturar_dados(base):
    """Dada uma base de dados, é realizada toda a estruturação das bases.
    Args:
        base: contém todos tweets no formato (texto,classe).
    Returns:
        base_final: conjunto de dados estruturados.
    """
    global palavras_sem_repeticao
    # Aplicar as funções previamente definidas
    frases_sem_pontuacao = remover_pontuacao(base)
    frases_sem_stopwords = remover_stopwords(frases_sem_pontuacao)
    frases_com_stemmer = aplicar_stemmer(frases_sem_stopwords)
    palavras_sem_classe = extrair_palavras(frases_com_stemmer)
    frequencia_palavras = aplicar_frequencia(palavras_sem_classe)
    palavras_sem_repeticao = extrair_palavras_unicas(frequencia_palavras)
    base_final = nltk.classify.apply_features(criar_caracteristicas, frases_com_stemmer)
    # Retornar os dados estruturados para serem utilizados pela função NLTK.
    return (base_final)

## 3. Fase de treino do algoritmo Naive Bayes

Nesta etapa será realizado o treino do algoritmo Naive Bayes, que irá gerar um modelo a ser utilizado na classificação de novas frases em positivo, negativo ou neutro.

## 3.1 Classificação do texto

O algoritmo de Naive Bayes realiza a análise estatística e monta uma tabela de probabilidade. Após isso, é criada a classificação dos registros.

O método ```train``` da classe ```NaiveBayesClassifier``` recebe como parâmetro a ```base_treino``` já estruturada (```base_final```) e realiza a etapa de construção da tabela de probabilidades. O método ```show_most_informative_features``` retorna os atributos (palavras) mais significativos.

Por exemplo: 

- ```dia = True positi : negati = 2.3:1.0``` - Neste exemplo de saída a probabilidade de a frase ser classificada como ```positivo``` quando a palavra "dia" estiver presente na frase (```True```) é 2.3 vezes maior do que negativo.

- ```am = False negati : positi = 1.6:1.0``` - Já neste exemplo, a probabilidade de a frase ser classificada como ```negativo``` quando a palavra ```am``` **não** estiver presente na frase (```False```) é 1.6 vezes maior do que positivo.

In [ ]:
base_final = estruturar_dados(base_treino)

In [ ]:
%%time
# Cria o classificador (tabela de probabilidade) com base no conjunto de treinamento
classificador = nltk.NaiveBayesClassifier.train(base_final)

In [ ]:
# Retorna as classes da base de dados (positivo, negativo, neutro)
print(classificador.labels())

In [ ]:
# Retorna os 10 atributos mais significativos
print(classificador.show_most_informative_features(10))

### 3.2 Testando o classificador

Para testar o classificador, iremos utilizar uma frase para verificar a classificação realizada, uma vez que a base de dados já foi pré-processada e treinada com a base de dados para treino.

Para que a nova frase seja classificada temos que realizar toda a fase de pré-processamento. Como criamos a função ```estruturar_dados``` podemos simplesmente utilizá-la :)

In [ ]:
base_teste[0]

In [ ]:
frase_teste = estruturar_dados([base_teste[0]])
print(frase_teste)


In [ ]:
print(frase_teste[0][0])
caracteristica_teste = frase_teste[0][0]



Quando chamamos o método ```classificador.classify``` com o parâmetro ```frase_teste```, o algoritmo classifica a frase como rótulo ```negativo```.

In [ ]:
# Realizar a classificação
print(classificador.classify(caracteristica_teste))


Para visualizar a distribuição de probabilidade utiliza-se o método ```prob_classify``` que mostra a porcentagem para cada uma das classes.

In [ ]:
# Retorna a classe e o valor da distribuição de probabilidade
distribuicao = classificador.prob_classify(caracteristica_teste)

# Para cada classe, verifica-se a probabilidade
for classe in distribuicao.samples():
    print('%s: %f' % (classe, distribuicao.prob(classe)))

## 4. Fase de teste do algoritmo Naive Bayes

Na etapa de teste, utiliza-se outro conjunto de dados, com o objetivo de testar o algoritmo de aprendizado de máquina com novas frases. Para tal, a base de dados deve conter frases diferentes da base de treinamento e sem a informação de sua classificação (positivo, negativo e neutro).

O algoritmo é executado novamente e o método ```classify.accuracy```  mostra a proximidade entre o percentual obtido experimentalmente e o valor verdadeiro da classificação das frases.

Passamos como parâmetro o classificador, que nada mais é do que uma tabela de probabilidade que o Naive Bayes gera, e a base de dados para teste.

In [ ]:
frases_teste = estruturar_dados(base_teste)

In [ ]:
print(frases_teste[:3])

O método ```classify.accuracy``` funciona da seguinte forma: ele submete todos os registros da base de teste ao classificador e o classificador gera uma classificação para cada um dos registros. Após isso, realiza uma comparação entre a classificação gerada e a classifcação que já tinha sido realizada na base de dados, e devolve a taxa de acerto.

In [ ]:
print(nltk.classify.accuracy(classificador, frases_teste))

Esse resultado, possibilita realizar algumas análises, tais como:

1. **Análise de cenáro**: O percentual de acerto do algoritmo é bom ou ruim? 
2. **Análise do numero de classes**: A probabilidade mínima aceitavel para o algoritmo ser melhor do que usar a aleatoriedade é que a acurácia seja no mínimo maior que 33.33%, ou seja, dividir 100% pela quantidade de classes.
3. **ZeroRules**: Nessa análise, estamos comparando o resultado obtido pelo sistema, com o método de classificar uma frase de acordo com a classe que possui maior quantidade de frases na base de dados de treino e teste. Por exemplo, dividimos a classe com maior número de registros pelo total de registros na base de dados (```459/1374 = 33,40%```). Desta forma, conclui-se que o sistema apresenta mais acertos do que classificar todas as novas frases nessa classe.

In [ ]:
res = {'positivo' : 0, 'negativo' : 0, 'neutro' : 0}
total = 0
for (texto, classe) in base_teste:
    if classe == 'positivo':
        res[classe] += 1
    elif classe == 'negativo':
        res[classe] += 1
    elif classe == 'neutro':
        res[classe] += 1
    total += 1
print(res)

In [ ]:
print(res['negativo']/total)

## 5. Extra - Visualização de erros do algoritmo

É possível visualizar a classe já pré-classificada, a classe que o algoritmo classificou e a frase vinculada ao erro gerado.

Por exemplo: ```positivo negativo {'trabalh': False, ... , 'precis': True,'ingress': True, 'estrag': False,...}```. Essa saída nos diz qual a classe correta, ou seja, aquela que está na base de dados para teste é ```positivo```. O algoritmo classificou como ```negativo``` e a frase vinculada a classificação possui os radicais ```precis``` e ```ingress```.

Para identificar corretamente os acertos e erros podemos:

In [ ]:
erros =[]
for (frase, classe) in frases_teste:
    resultado = classificador.classify(frase)
    
    if resultado != classe:
        erros.append((classe, resultado, frase))
        
        

Desta forma, é possível verificar a porcentagem de erro e acerto realizado no conjunto de dados de teste.

In [ ]:
tamanho_base_teste = len(frases_teste)
quantida_erros = len(erros)

porcentagem_erros = (quantida_erros * 100) / tamanho_base_teste
porcentagem_acertos = 100 - porcentagem_erros

print("O algoritmo classificou {:.4}% das frases corretamente".format(porcentagem_erros))
print("O algoritmo classificou {:.4}% das frases incorretamente".format(porcentagem_acertos))



### 5.1 Matriz de confusão

Outra forma de visualização de erros e acertos, é a construção da matriz de confusão:

- Primeiramente importamos o pacote do ```nltk``` com a função da matriz de confusão.
- Criamos duas listas, uma com o resultado ```esperado``` e outra com o resultado ```previsto```, sendo que o esperado é o resultado desejado como resposta, e o previsto é de fato a classificação realizada.
- A saída do algoritmo mostra uma matriz com linhas que represantam o esperado e colunas que representam o previsto.
- A diagonal principal indica a quantidade de acertos de cada classe.

Esses resultados apresentam as classes que o algoritmo está mais errando e/ou acertando, sendo assim, é possível tomar decisões para melhorar a implementação da base de dados, bem como alguns parâmetros de otimização do algoritmo.

In [ ]:
from nltk.metrics import ConfusionMatrix

In [ ]:
esperado = []
previsto = []
for (frase, classe) in frases_teste:
    resultado = classificador.classify(frase)
    previsto.append(resultado)
    esperado.append(classe)

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)




### 6. Utilizaremos ambos os conjuntos de dados para criar o modelo final

Uma vez verificado a acuracia e as métricas em nosso conjunto de dados de treino e teste, podemos unificar esses dois conjuntos de dados para utilizar todas as palavras com o objetivo de maximar a curva de aprendizado do algoritmo.


In [ ]:
print(type(base_teste), type(base_treino))

In [ ]:
conj_final = estruturar_dados(base_teste+base_treino)

In [ ]:
%%time
# Cria o classificador (tabela de probabilidade) com base no conjunto de treinamento
classificador_final = nltk.NaiveBayesClassifier.train(conj_final)

In [ ]:
# Retorna as classes da base de dados (positivo, negativo, neutro)
print(classificador_final.labels())

In [ ]:
# Retorna os 10 atributos mais significativos
print(classificador_final.show_most_informative_features(10))

### 7. Salvar o modelo criado para realizar a análise de sentimento nos videos do Youtube

In [ ]:
import pickle

In [ ]:
def salvar_modelo(modelo, nome_arquivo):
    nome = str(nome_arquivo) + ".pickle"
    try:
        salvar_modelo = open(nome,"wb")
        pickle.dump(modelo, salvar_modelo)
        salvar_modelo.close()
        return True
    except Exception as e:
        return e

In [ ]:
if salvar_modelo(classificador,'naivebayes'):
    print("Modelo salvo para ser utilizado no futuro :)")
    

In [ ]:
print("Tempo total para executar esse notebook foi de {} segundos".format(time() - ti))